In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 01: Single Agent

This notebook demonstrates a basic financial assistant agent that uses web search to answer financial questions.
The agent "collaborates" with itself using the ReAct pattern.

## How the Agent Works: ReAct Pattern

The agent follows the **ReAct** (Reasoning + Acting) pattern, which combines reasoning and acting in an iterative loop:

```mermaid
graph TD
    Start([👤 User Question 👤]) --> Reason[🤖 Agent: Analyze & Reason<br/>- Understand question<br/>- Plan approach<br/>- Decide on tools]
    Reason --> Decision{Need<br/>Tools?}
    
    Decision -->|No| Response[📝 Generate<br/>Final Response]
    Decision -->|Yes| CallTool[🔧 Call Tool<br/>Web Search]
    
    CallTool --> Results[📊 Receive<br/>Tool Results]
    Results --> React[🤖 Agent: React<br/>- Process results<br/>- Integrate information<br/>- Decide next step]
    
    React --> Decision
    
    Response --> End([✅ Return Answer&nbsp;<br/>to User])
    
    style Start fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style Reason fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Decision fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style CallTool fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style Results fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style React fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Response fill:#e0f2f1,stroke:#004d40,stroke-width:2px
    style End fill:#e1f5ff,stroke:#01579b,stroke-width:3px
```

**Key Points:**
- **Reasoning**: The agent analyzes the question and decides what actions to take
- **Acting**: The agent calls tools (like web search) to gather information
- **Iteration**: This process repeats until the agent has enough information (shown by the loop back from React to Decision)
- **Reflection**: After each tool call, the agent reflects on the results and decides next steps


In [ ]:
# Setup: Import necessary modules
from IPython.display import Markdown, display
from pydantic_ai import Agent
from common.tools import web_search_tool
from common.utils import (
    create_agent_model,
    setup_logging,
    setup_logfire,
    get_current_date,
)

# Initialize logging
setup_logging()

# Initialize Logfire instrumentation (for observability)
setup_logfire(
    service_name="financial-assistant-lecture01",
    start_message="🚀 Lecture 01 - Financial Assistant Notebook Started",
)

In [ ]:
# Create the financial assistant agent
financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    system_prompt=f"""
You are a knowledgeable financial assistant. Your role is to help users with financial questions,
investment advice, market analysis, and general financial guidance.
IMPORTANT CONTEXT:
- Today's date is {get_current_date()}
- Always provide information in the context of this current date


When answering questions:
1. Use the web_search tool to get current, up-to-date financial information when relevant
2. Combine web search results with your financial knowledge
3. Provide clear, actionable advice
4. Always mention when information is based on web search vs. general knowledge
5. Include appropriate disclaimers for investment advice
6. Be helpful but remind users to consult with financial professionals for major decisions

Keep your responses informative but concise, and always prioritize accuracy and user safety.
""",
)

In [4]:
# Ask a question
question = "What is the current price of Bitcoin?"
result = await financial_agent.run(question)

# Display with nice Markdown formatting
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

**Question:** What is the current price of Bitcoin?

---

Based on today's latest market data (as of January 5, 2026), the current price of Bitcoin (BTC) is approximately between \$91,800 and \$92,900 USD, with some variation across exchanges.

Sources:
- Phemex: \$91,163.00
- Coinbase: \$92,914.03
- CoinGecko: \$91,837.94
- Delta: \$92,655.58

Please note that cryptocurrency prices fluctuate rapidly, and the actual price may vary depending on the platform you use. This information was gathered via web search for accuracy.

If you are planning to trade or invest, always check the price at your chosen exchange before making any financial decisions.

In [5]:
# Try another question
question = "Find 3 trending stocks and for each one provide an investor's report"
result = await financial_agent.run(question)

# Display with nice Markdown formatting
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
# Escape dollar signs to prevent LaTeX interpretation
display(Markdown(result.output.replace("$", "\\$")))

**Question:** Find 3 trending stocks and for each one provide an investor's report

---

Based on up-to-date web search (January 2026), three trending stocks are:

1. Advanced Micro Devices (AMD)
2. NVIDIA (NVDA)
3. Meta Platforms (META)

Below are investor reports for each:

---

### 1. Advanced Micro Devices (AMD)

**Overview:**  
AMD is at the forefront of the AI revolution, benefiting from surging demand for GPUs and AI chips in both consumer electronics and data centers. The company is widely regarded as a compelling AI trade for 2026.

**Recent Performance:**  
- AMD stock has outperformed broader indices, with robust momentum heading into 2026.
- Growth prospects are bolstered by continued expansion in AI, gaming, and server segments.

**Outlook:**  
- Market analysts project strong revenue and earnings growth, driven by AI chip sales and strategic partnerships.
- Risks include fierce competition (notably from NVIDIA), supply chain challenges, and pricing pressures.

**Recommendation:**  
AMD presents a solid growth opportunity for investors seeking exposure to the AI and semiconductor sectors. However, potential investors should monitor technological shifts and competitive threats.

---

### 2. NVIDIA (NVDA)

**Overview:**  
NVIDIA remains a dominant force in AI and data center markets, with its RTX and data-center hardware seeing explosive demand.

**Recent Performance:**  
- NVIDIA’s stock price has experienced significant appreciation, making it one of the market leaders in 2026.
- Strategic deals (e.g., large AI hardware contracts) and expansion into cloud AI services underpin growth.

**Outlook:**  
- Analysts predict NVIDIA could drive the next major market rally, given its assets in AI architecture and expanding commercial applications.
- Risks include regulatory headwinds, potential tech sector pullbacks, and pressure from emerging competitors.

**Recommendation:**  
NVIDIA is a top pick for those wanting AI-driven growth and exposure to large-scale data center upgrades. Caution is advised concerning valuation risks after the strong recent rally.

---

### 3. Meta Platforms (META)

**Overview:**  
Meta Platforms (formerly Facebook) continues to grow its social media ecosystem and invest heavily in AI-driven ad tech and virtual reality.

**Recent Performance:**  
- META stock is trending due to strong ad revenue, new product launches, and expanding AI integration across its platforms.
- The company is capitalizing on user engagement and monetization of harder-to-reach digital audiences.

**Outlook:**  
- Continued investment in metaverse technologies and AI is expected to drive future growth.
- Risks involve regulatory scrutiny, changing user habits, and rising competition in social and AI spaces.

**Recommendation:**  
META is well-positioned for digital ad and tech innovation-driven growth. Investors should monitor regulatory developments and evolving consumer trends closely.

---

_Disclaimer: This report is based on recent news and market analysis. All investing inherently involves risk. Consider consulting with a qualified financial advisor before making major investment decisions._